In [5]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from osgeo import ogr, gdal
from sklearn.utils import shuffle
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
from tensorflow.keras.layers import Input, Activation, Dense, Conv2D, MaxPool2D, Dropout, concatenate, UpSampling2D, Flatten
from tensorflow.keras.models import Model, load_model, Sequential
import tensorflow.keras.backend as K
#from keras.layers.merge import concatenate, add

import os
from skimage.util.shape import view_as_windows
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [6]:
gpus= tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [7]:
def load_tiff_image(image):
    print (image)
    gdal_header = gdal.Open(image)
    img = gdal_header.ReadAsArray()
    return img

def normalization(image):
    return np.array(image)/127.5 - 1.

def extract_patches(image, reference, patch_size, stride):
    patches_out = []
    label_out = []
    window_shape = patch_size
    window_shape_array = (window_shape, window_shape, image.shape[2])
    window_shape_ref = (window_shape, window_shape)
    patches_array = np.array(view_as_windows(image, window_shape_array, step = stride))
    
    patches_ref = np.array(view_as_windows(reference, window_shape_ref, step = stride))
    
    print(patches_array.shape)
    num_row,num_col,p,row,col,depth = patches_array.shape
    
    patches_array = patches_array.reshape(num_row*num_col,row,col,depth)
    print(patches_array.shape)
    patches_ref = patches_ref.reshape(num_row*num_col,row,col)
    print(patches_ref.shape)
    
    return patches_array, patches_ref

In [8]:
root_path = './dataset'
# Load images
img_train_path = 'clipped_raster_004_66_2018.tif'
img_train = load_tiff_image(os.path.join(root_path, img_train_path)).astype(np.float32)
img_train = img_train.transpose((1,2,0))
# Normalizes the image
#img_train_normalized = normalization(img_train)
# Transform the image into W x H x C shape
#img_train_normalized = img_train_normalized.transpose((1,2,0))
print(img_train.shape)

# Load reference
img_train_ref_path = 'labels/binary_clipped_2013_2018.tif'
img_train_ref = load_tiff_image(os.path.join(root_path, img_train_ref_path))
print(img_train_ref.shape)
# img_train_ref = img_train_ref.transpose((1,2,0))
# print(img_train_ref.shape)

# Load Mask area
img_mask_ref_path = 'mask_ref.tif'
img_mask_ref = load_tiff_image(os.path.join(root_path, img_mask_ref_path))
img_mask_ref = np.copy(img_mask_ref[:6102, :6613])
print(img_mask_ref.shape)

./dataset/clipped_raster_004_66_2018.tif


AttributeError: 'NoneType' object has no attribute 'ReadAsArray'

In [5]:
plt.imshow(img_mask_ref)

In [6]:
# Area preta
print(img_mask_ref[0, 0])
# Area amarela
print(img_mask_ref[3000, 4000])

-99.0
-1.0


In [12]:
# Area preta
print(img_train_ref[0, 0])
# Area amarela
print(img_train_ref[3000, 4000])
h,w = img_train_ref.shape
unique, counts = np.unique(img_train_ref, return_counts=True)
counts_dict = dict(zip(unique, counts))
print(counts_dict)

0.0
0.0
{0.0: 40287112, 1.0: 65414}


In [25]:
# Area preta
print(img_train[0, 0, :])
# Area amarela
print(img_train[3000, 4000, :])

[0. 0. 0. 0. 0. 0. 0.]
[ 9178.  8222.  7599.  6705. 16499. 10355.  7058.]


In [20]:
shape = img_mask_ref.shape
patches_train = []
patches_train_ref = []
new_img = np.array(())
patch_size = 128
stride = patch_size // 1
#stride = 1
num_patches_h = shape[0] // stride
num_patches_w = shape[1] // stride
print(f'Patche: {patch_size}')
print(f'Stride: {stride}')
print(f'Num patches h: {num_patches_h}')
print(f'Num patches w: {num_patches_w}')
for i in range(num_patches_h):
    for j in range(num_patches_w):
        #print(j*patch_size)
        patch = np.copy(img_mask_ref[i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size])
        patch_train_ref = img_train_ref[i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size]
        patch_train = img_train[i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size]
        unique, counts = np.unique(patch_train_ref, return_counts=True)
        counts_dict = dict(zip(unique, counts))
        if 1 in counts_dict.keys():
            total_pixels = counts_dict[0] + counts_dict[1]
            if np.all(patch == -1) == True and counts_dict[1]/total_pixels >= 0.2:
                #print(counts_dict[1]/total_pixels)
                patches_train.append(patch_train)
                patches_train_ref.append(patch_train_ref)
            else:
                pass
print(len(patches_train))
print(len(patches_train_ref))
#patches=np.asarray(patches)

Patche: 128
Stride: 128
Num patches h: 47
Num patches w: 51
0
0


In [25]:
# Load reference
img_train_ref_path = '/media/thimabru/ssd/Deep_Learning_aula/Homework3_ELE2765/Reference_Train.tif'
img_train_ref = load_tiff_image(os.path.join(root_path, img_train_ref_path))
img_train_ref = img_train_ref.transpose((1,2,0))
print(img_train_ref.shape)
w = img_train_ref.shape[0]
h = img_train_ref.shape[1]
c = img_train_ref.shape[2]
#binary_img_train_ref = np.zeros((1,w,h))
binary_img_train_ref = np.full((w,h), -1)
label_dict = {'(255, 255, 255)': 0, '(0, 255, 0)': 1, '(0, 255, 255)': 2, '(0, 0, 255)': 3, '(255, 255, 0)': 4}
for i in range(w):
    for j in range(h):
        r = img_train_ref[i][j][0]
        g = img_train_ref[i][j][1]
        b = img_train_ref[i][j][2]
        rgb = (r,g,b)
        rgb_key = str(rgb)
        binary_img_train_ref[i][j] = label_dict[rgb_key]

/media/thimabru/ssd/Deep_Learning_aula/Homework3_ELE2765/Reference_Train.tif
(2565, 1919, 3)


In [32]:
shape = img_mask_ref.shape
patches = []
new_img = np.array(())
patch_size = 128
stride = patch_size // 4
#stride = 1
num_patches_h = shape[0] // stride
num_patches_w = shape[1] // stride
print(f'Patche: {patch_size}')
print(f'Stride: {stride}')
print(f'Num patches h: {num_patches_h}')
print(f'Num patches w: {num_patches_w}')
for i in range(num_patches_h):
    for j in range(num_patches_w):
        #print(j*patch_size)
        patch = np.copy(img_mask_ref[i*stride:(i+1)*stride, j*stride:(j+1)*stride])
        if np.all(patch == -1) == True:
#             #print(i,j)
#             print(len(patch.tolist()))
#             print(patch.shape)
            patches.append(patch)
        else:
            pass
print(len(patches))
#patches=np.asarray(patches)

Patche: 128
Stride: 32
Num patches h: 190
Num patches w: 206
28794


In [33]:
print(patches[0].shape)

(32, 32)


In [35]:
print(patches)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [34]:
shape = img_train_ref.shape
print(shape)

(2565, 1919, 3)


In [42]:
label_dict = {}
for i in range(shape[0]):
    for j in range(shape[1]):
        pixel = img_train_ref[i, j]
        pixel_key = str(pixel)
        if pixel_key not in label_dict.keys():
            label_dict[pixel_key] = 0
        else:
            label_dict[pixel_key] += 1
        #print(img_train_ref[i, j])
        #print(len(img_train[i, j]))

In [43]:
print(label_dict)

{'0.0': 38932622, '1.0': 1419902}


In [10]:
shape = img_train_normalized.shape
print(shape)
label_dict = {}
l = []
for i in tqdm(range(shape[0])):
    for j in range(shape[1]):
        pixels = img_train_normalized[i, j, :].tolist()
        if pixels not in l:
            print(pixels)
            l.append(pixels)
        
        #print(type(pixels))
#         if pixels.any() not in l:
#             l.append(pixels)
#             print(pixels)
#         if None in pixel:
#             print(pixel)
        #print(img_train_ref[i, j])
        #print(len(img_train[i, j]))



  0%|          | 0/6102 [00:00<?, ?it/s]

  0%|          | 12/6102 [00:00<00:51, 117.61it/s]

(6102, 6613, 7)
[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]
[75.23529052734375, 67.45490264892578, 61.0313720703125, 53.0, 126.57646942138672, 74.3490219116211, 52.2156867980957]
[75.02352905273438, 67.22744750976562, 60.427452087402344, 52.60784149169922, 128.2392120361328, 73.26667022705078, 50.81960678100586]
[75.05490112304688, 67.5568618774414, 60.66274642944336, 53.13333511352539, 121.43921661376953, 71.93333435058594, 51.525489807128906]
[75.44705963134766, 67.78431701660156, 61.45490264892578, 53.69804000854492, 126.65489959716797, 75.0313720703125, 52.16862869262695]
[74.92156982421875, 67.26667022705078, 61.070587158203125, 52.3725471496582, 126.45882415771484, 74.86666870117188, 51.603919982910156]
[74.77255249023438, 66.86666870117188, 60.05882263183594, 51.80784225463867, 119.32941436767578, 73.66666412353516, 51.01568603515625]
[74.67842864990234, 67.0, 60.51372528076172, 52.02745056152344, 126.92941284179688, 74.52156829833984, 51.25882339477539]
[74.65489959716797, 66.82



  0%|          | 19/6102 [00:00<01:03, 96.47it/s] 

  0%|          | 24/6102 [00:00<01:26, 70.13it/s]


[74.41960906982422, 66.70195770263672, 59.79215621948242, 51.73725509643555, 121.65882110595703, 75.07058715820312, 51.8156852722168]
[74.09803771972656, 66.40392303466797, 59.35293960571289, 51.27450942993164, 118.17646789550781, 73.01568603515625, 50.85097885131836]
[74.14509582519531, 66.33333587646484, 60.04313659667969, 51.67451095581055, 139.062744140625, 73.80000305175781, 50.64706039428711]
[74.12940979003906, 66.51372528076172, 59.80784225463867, 52.23921585083008, 129.7372589111328, 73.80784606933594, 51.509803771972656]
[74.57646942138672, 67.13333129882812, 60.51372528076172, 52.98431396484375, 121.96470642089844, 75.2196044921875, 53.298038482666016]
[75.75294494628906, 68.51372528076172, 61.7529411315918, 54.52941131591797, 118.95294189453125, 80.4117660522461, 56.9058837890625]
[74.58431243896484, 67.08627319335938, 60.32548904418945, 52.521568298339844, 110.48235321044922, 72.64705657958984, 51.8156852722168]
[74.21568298339844, 66.57646942138672, 59.37647247314453, 51



  0%|          | 29/6102 [00:00<01:55, 52.50it/s]

...

[74.23921203613281, 66.45882415771484, 59.250980377197266, 51.525489807128906, 114.5999984741211, 70.38823699951172, 50.10588073730469]
[74.97647094726562, 67.47058868408203, 61.0313720703125, 53.36862564086914, 118.12940979003906, 76.4745101928711, 53.29019546508789]
[74.8431396484375, 67.29019927978516, 61.27450942993164, 53.51764678955078, 129.81568908691406, 76.6313705444336, 53.345096588134766]
[74.70980072021484, 67.10195922851562, 60.69411849975586, 52.72549057006836, 128.7490234375, 78.7254867553711, 53.16470718383789]
[74.60784149169922, 66.811767578125, 60.01960754394531, 52.2156867980957, 129.8941192626953, 75.28235626220703, 51.63529586791992]
[75.23529052734375, 67.31372833251953, 60.64706039428711, 52.992156982421875, 123.94117736816406, 77.20391845703125, 53.04705810546875]
[74.98431396484375, 67.20391845703125, 60.1843147277832, 52.31764602661133, 121.47843170166016, 74.82745361328125, 51.89411926269531]
[74.75686645507812, 66.89019775390625, 59.61176300048828, 51.72941

[75.32157135009766, 67.40784454345703, 60.30195999145508, 52.458824157714844, 125.26667022705078, 73.6196060180664, 50.701961517333984]
[75.18038940429688, 67.24313354492188, 60.66274642944336, 52.380393981933594, 133.72157287597656, 75.01568603515625, 51.117645263671875]
[75.14117431640625, 67.24313354492188, 61.219608306884766, 52.57646942138672, 137.9176483154297, 76.99215698242188, 51.77647018432617]
[75.22744750976562, 67.2745132446289, 60.929412841796875, 52.57646942138672, 133.5725555419922, 74.59215545654297, 51.14902114868164]
[75.23529052734375, 67.19607543945312, 60.48235321044922, 52.38823699951172, 128.79608154296875, 73.5882339477539, 50.85097885131836]
[74.85098266601562, 66.78823852539062, 59.42353057861328, 51.478431701660156, 117.66666412353516, 69.39215850830078, 49.33725357055664]
[74.7803955078125, 66.74117279052734, 59.7843132019043, 51.71372604370117, 124.79608154296875, 73.83137512207031, 51.156864166259766]
[75.10980224609375, 66.99215698242188, 60.160785675048

  1%|▏         | 87/6102 [00:13<51:36,  1.94it/s]

[74.21568298339844, 66.67058563232422, 60.20784378051758, 51.92549133300781, 134.52157592773438, 79.91764831542969, 53.32156753540039]
[74.6627426147461, 67.00784301757812, 60.17647171020508, 52.75686264038086, 126.46666717529297, 78.5058822631836, 53.054901123046875]
[74.33333587646484, 66.89804077148438, 60.67843246459961, 52.35686111450195, 128.47450256347656, 76.12940979003906, 52.49803924560547]
[74.74901580810547, 67.02352905273438, 60.46666717529297, 52.60784149169922, 129.94117736816406, 77.50196075439453, 52.858821868896484]
[74.30196380615234, 66.63921356201172, 60.568626403808594, 52.17647171020508, 130.44313049316406, 74.88235473632812, 51.68235397338867]
[74.4117660522461, 66.70980072021484, 60.615684509277344, 52.780391693115234, 130.30979919433594, 75.53333282470703, 52.2470588684082]
[74.5058822631836, 66.76470947265625, 60.43529510498047, 52.639217376708984, 125.80000305175781, 77.04705810546875, 53.19607925415039]
[74.3176498413086, 66.7254867553711, 60.63921737670898

KeyboardInterrupt: 

In [11]:
print(label_dict)

{}
